In [2]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
from sklearn.preprocessing import LabelEncoder, StandardScaler 
import os
load_dotenv()

OUTPUT_DIR = "output/"

In [3]:
engine = create_engine(os.environ.get("DATABASE_URL"))
engine.connect()

## Transform

1. Start with `match_players_df`
2. From `match_round_plant_df`, get total number of plants per match per player/agent
3. From `match_round_defuse_df`, get total number of defuses per match per player/agent
4. From `match_rounds_df`, get `winning_team`. Match `match_id` and `round_num` to  `match_round_player_stats_df`, get total number of rounds won in that match of that player
5. From `match_df`, get the `map_id` and `queue_id` for each match
6. From `map_df`, get `name` using the `map_id`
7. Join columns from 2, 3, 4, and 6.
8. From `agent_df`, get the agent names.


In [4]:
match_players_df = pd.read_sql("SELECT * FROM match_players;", engine)
match_round_plant_df = pd.read_sql("SELECT * FROM match_round_plant;", engine)
match_round_defuse_df = pd.read_sql("SELECT * FROM match_round_defuse;", engine)


In [5]:
defuses_per_match = match_round_defuse_df.groupby(['match_id', 'player_puuid', 'agent_id'])['round_num'].count().reset_index()
defuses_per_match = defuses_per_match.rename(columns={'round_num': 'total_defuses'})

In [6]:
match_rounds_df = pd.read_sql("SELECT * FROM match_rounds;", engine)
match_round_player_stats_df = pd.read_sql("SELECT * FROM match_round_player_stats;", engine)

rounds_data = match_rounds_df[['match_id', 'round_num', 'winning_team']]
total_rounds_per_match = match_rounds_df.groupby('match_id')['round_num'].count().reset_index()
total_rounds_per_match.rename(columns={'round_num': 'total_rounds'}, inplace=True)

player_round_data = pd.merge(match_round_player_stats_df, rounds_data,
                              on=['match_id', 'round_num'], how='left')
player_round_data['round_win'] = (player_round_data['winning_team'] == player_round_data['player_team']).astype(int)

# rounds_won_per_match = player_round_data.groupby(['match_id', 'player_puuid', 'agent_id'])['round_win'].sum().reset_index()
# rounds_won_per_match = rounds_won_per_match.rename(columns={'round_win': 'rounds_won'})

plants_per_match = match_round_plant_df.groupby(['match_id', 'player_puuid', 'agent_id'])['round_num'].count().reset_index()
plants_per_match = plants_per_match.rename(columns={'round_num': 'total_plants'})

In [7]:
match_df = pd.read_sql("SELECT * FROM match", engine)
map_df = pd.read_sql("SELECT * FROM map", engine)

map_ids = match_df[['id', 'map_id', 'queue_id']]
map_ids = map_ids.rename(columns={'id': 'match_id'})

map_names = map_df[['id', 'name']]
map_names = map_names.rename(columns={'id': 'map_id'})

map_mapping = pd.merge(map_ids, map_names, on='map_id', how='left')
map_mapping = map_mapping.drop(columns={'map_id'})

In [8]:
final_df = match_players_df.copy()

final_df = pd.merge(final_df, total_rounds_per_match, on='match_id', how='left')
final_df = pd.merge(final_df, plants_per_match, on=['match_id', 'player_puuid', 'agent_id'], how='left')
final_df = pd.merge(final_df, defuses_per_match, on=['match_id', 'player_puuid', 'agent_id'], how='left')
# final_df = pd.merge(final_df, rounds_won_per_match, on=['match_id', 'player_puuid', 'agent_id'], how='left')
# final_df[['total_plants', 'total_defuses', 'rounds_won']] = final_df[['total_plants', 'total_defuses', 'rounds_won']].fillna(0).astype(int)

final_df[['total_plants', 'total_defuses']] = final_df[['total_plants', 'total_defuses']].fillna(0).astype(int)


final_df = pd.merge(final_df, map_mapping, on='match_id', how='left')

In [9]:
agent_df = pd.read_sql('SELECT * FROM agent', engine)

agent_names = agent_df[['id', 'name']]
agent_names = agent_names.rename(columns={'id': 'agent_id'})

final_df = pd.merge(final_df, agent_names, on=['agent_id'], how='left')
final_df = final_df.rename(columns={'name_x': 'map_name', 'name_y': 'agent_name'})

In [10]:
match_teams_df = pd.read_sql("SELECT * FROM match_teams;", engine)

final_df = final_df.merge(match_teams_df, on=['match_id', 'team_id'], how='inner')
final_df

,match_id,player_puuid,team_id,platform,party_id,agent_id,stats_score,stats_kills,stats_deaths,stats_assists,...,economy_loadout_value_average,total_rounds,total_plants,total_defuses,queue_id,map_name,agent_name,won,rounds_won,rounds_lost
0,7feb4a95-efc8-4a6f-97e1-b6d2dbf4bcf0,c49d08a1-f0c6-5c1d-8039-e651f790f31d,Red,pc,9127aec2-14a1-4ddd-8c1a-5fc469f35215,320b2a48-4d9b-a075-30f1-1f93a9b638fa,3847,13,19,7,...,3443.7500,24.0,2,0,competitive,Ascent,Sova,False,11,13
1,7feb4a95-efc8-4a6f-97e1-b6d2dbf4bcf0,0bead0a5-8154-5854-96b1-03c81facb0ab,Blue,pc,96f405bb-a525-4667-93ed-9fe260a56ffd,a3bfb853-43b2-7238-a4f1-ad90e9e46bcc,5800,20,19,8,...,3743.7500,24.0,1,0,competitive,Ascent,Reyna,True,13,11
2,7feb4a95-efc8-4a6f-97e1-b6d2dbf4bcf0,b258abbc-d428-5dff-bb3c-d0224afffb5b,Red,pc,80813d58-0672-492c-ad6e-4447fb79a990,117ed9e3-49f3-6512-3ccf-0cada7e3823b,4344,16,20,5,...,3250.0000,24.0,0,0,competitive,Ascent,Cypher,False,11,13
3,7feb4a95-efc8-4a6f-97e1-b6d2dbf4bcf0,6878729b-2ad5-580c-b11f-a2beacc42e6b,Red,pc,9127aec2-14a1-4ddd-8c1a-5fc469f35215,add6443a-41bd-e414-f6ad-e58d267f4e95,6164,21,19,1,...,3462.5000,24.0,0,0,competitive,Ascent,Jett,False,11,13
4,7feb4a95-efc8-4a6f-97e1-b6d2dbf4bcf0,7ae15c49-1e2c-5681-b033-1d691ed66896,Blue,pc,3341a0d0-4d8b-4d52-a119-478127299895,320b2a48-4d9b-a075-30f1-1f93a9b638fa,4681,17,16,3,...,3625.0000,24.0,2,0,competitive,Ascent,Sova,True,13,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611691,c8958932-796f-4813-acc8-3a5edfb97fe2,9d0c95e6-3dcc-5dd9-981d-c451ae720bef,Blue,pc,65144807-d381-4f40-93db-d69085fb4897,1dbf2edd-4729-0984-3115-daa5eed44993,7382,25,24,9,...,4125.0000,30.0,1,0,competitive,Sunset,Clove,False,15,15
611692,c8958932-796f-4813-acc8-3a5edfb97fe2,7c9de286-2622-5472-a5c9-74dabcca7645,Blue,pc,f5fde002-6245-483f-a6a5-c946f1ba3c9b,320b2a48-4d9b-a075-30f1-1f93a9b638fa,6372,22,22,4,...,3696.6667,30.0,7,2,competitive,Sunset,Sova,False,15,15
611693,c8958932-796f-4813-acc8-3a5edfb97fe2,8b84a35e-ec3c-5948-9af5-97f028911f7d,Red,pc,d93f0f12-69f1-449b-a4ca-6505980223d9,1dbf2edd-4729-0984-3115-daa5eed44993,8078,29,23,11,...,4003.3333,30.0,0,0,competitive,Sunset,Clove,False,15,15
611694,c8958932-796f-4813-acc8-3a5edfb97fe2,5233e820-50a6-5b7d-96fc-c689ff080f66,Red,pc,a148eb68-11b6-4e65-994f-bed40e9a84d9,22697a3d-45bf-8dd7-4fec-84a9e28c69d7,9031,32,19,2,...,4303.3335,30.0,0,0,competitive,Sunset,Chamber,False,15,15


## Pre-Processing

1. Drop the following columns:
- `queue_id`: Everything except `competitive`, `premier`, `unrated`. Focus only on versus modes*
- Columns:`match_id`, `player_puuid`, `team_id`, `platform`, `party_id`, `agent_id`, `behavior_rounds_in_spawn`, `behavior_afk_rounds`, `session_playtime_`,`behavior_friendly_fire_incoming`, `behavior_friendly_fire_outgoing`.  Above values do not reflect the impact of agent abilities
- `NaN`s

2. Handle Categorical Data
- Create dictionary integer assignment for `queue_id`, `map_name`, `agent_name`

3. Derived Features
- KDA Ratio = $\frac{(kill + assist)}{death}$. *The higher the better*
- Damage per Kill = $\frac{damage\space dealt}{kills}$
- ADR = Average Damage per Round
- ACS = Average Combat Score = $\left(\frac{stat\space score}{rounds}\right)$
- Overall ability cast per round = $\frac{(ability1 + ability2 + grenade + ultimate)}{ Rounds}$
- Win percentage

4. Standardize
- `StandardScaler()` on numerical features, exclude `queue_id`, `map_name`, `agent_name`

5. Checking Rounds Won per Agent, Correlation

### Drop `NaN`s and irrelevant columns

In [11]:
df = final_df

df_clean = df.drop(['match_id', 'player_puuid', 'team_id', 'platform', 'party_id', 'agent_id', 'behavior_rounds_in_spawn', 'behavior_afk_rounds', 'behavior_friendly_fire_incoming', 'behavior_friendly_fire_outgoing', 'session_playtime_in_ms'], axis=1)
df_clean = df_clean.dropna()
df_clean = df_clean.reset_index(drop=True)
print(df_clean.isna().sum())

stats_score                      0
stats_kills                      0
stats_deaths                     0
stats_assists                    0
stats_headshots                  0
stats_bodyshots                  0
stats_legshots                   0
stats_damage_dealt               0
stats_damage_received            0
ability_casts_grenade            0
ability_casts_ability1           0
ability_casts_ability2           0
ability_casts_ultimate           0
tier_id                          0
account_level                    0
economy_spent_overall            0
economy_spent_average            0
economy_loadout_value_overall    0
economy_loadout_value_average    0
total_rounds                     0
total_plants                     0
total_defuses                    0
queue_id                         0
map_name                         0
agent_name                       0
won                              0
rounds_won                       0
rounds_lost                      0
dtype: int64


In [12]:
df_clean = df_clean[df_clean['queue_id'].isin(['competitive', 'premier', 'unrated'])]

### Handle Categorical Data

In [13]:
queue_id_dict = {
    'competitive': 0,
    'premier': 1,
    'unrated': 2}

map_dict = {
    'Abyss': 0,
    'Split': 1,
    'Haven': 2,
    'Bind': 3,
    'Pearl': 4,
    'Ascent': 5,
    'Sunset': 6,
    'Fracture': 7,
    'Icebox': 8,
    'Lotus': 9,
    'Breeze': 10
}

agent_dict = {
    'Omen': 0,
    'Sova': 1,
    'Jett': 2,
    'Chamber': 3,
    'Reyna': 4,
    'Killjoy': 5,
    'Astra': 6,
    'Cypher': 7,
    'Neon': 8,
    'Gekko': 9,
    'Clove': 10,
    'Vyse': 11,
    'Skye': 12,
    'Iso': 13,
    'Yoru': 14,
    'Raze': 15,
    'Phoenix': 16,
    'Deadlock': 17,
    'Harbor': 18,
    'KAY/O': 19,
    'Viper': 20,
    'Sage': 21,
    'Brimstone': 22,
    'Fade': 23,
    'Breach': 24
}

In [14]:
# Convert 'queue_id' to numeric using the queue_id_dict
df_clean['queue_id_numeric'] = df_clean['queue_id'].map(queue_id_dict)

# Convert 'map_name' to numeric using the map_dict
df_clean['map_name_numeric'] = df_clean['map_name'].map(map_dict)

# Convert 'agent_name' to numeric using the agent_dict
df_clean['agent_name_numeric'] = df_clean['agent_name'].map(agent_dict)

# Drop the original categorical columns
df_clean.drop(['queue_id', 'map_name', 'agent_name'], axis=1, inplace=True)

### Add Derived Features

In [15]:
df_clean['KDA Ratio'] = (df_clean['stats_kills'] + df_clean['stats_assists'])/ (df_clean['stats_deaths'] +1)
df_clean['ADR'] = df_clean['stats_damage_dealt'] / df_clean['total_rounds']
df_clean['ACS'] = df_clean['stats_score'] / df_clean['total_rounds']
df_clean['Overall Ability Cast'] = (df_clean['ability_casts_grenade']+df_clean['ability_casts_ability1']+df_clean['ability_casts_ability2']+df_clean['ability_casts_ultimate']) / df_clean['total_rounds']
df_clean['Win Ratio'] = df_clean['rounds_won']/df_clean['total_rounds']

In [16]:
def calculate_damage_per_kill(row):
    if row['stats_kills'] == 0:
        return 0
    else:
        return row['stats_damage_dealt'] / row['stats_kills']

df_clean['damage_per_kills'] = df_clean.apply(calculate_damage_per_kill, axis=1)
df_clean

,stats_score,stats_kills,stats_deaths,stats_assists,stats_headshots,stats_bodyshots,stats_legshots,stats_damage_dealt,stats_damage_received,ability_casts_grenade,...,rounds_lost,queue_id_numeric,map_name_numeric,agent_name_numeric,KDA Ratio,ADR,ACS,Overall Ability Cast,Win Ratio,damage_per_kills
0,3847,13,19,7,12,22,0,2589,3968,15.0,...,13,0,5,1,1.000000,107.875000,160.291667,2.958333,0.458333,199.153846
1,5800,20,19,8,19,23,3,3788,3433,13.0,...,11,0,5,4,1.400000,157.833333,241.666667,1.291667,0.541667,189.400000
2,4344,16,20,5,7,44,11,2717,3462,28.0,...,13,0,5,7,1.000000,113.208333,181.000000,3.291667,0.458333,169.812500
3,6164,21,19,1,16,24,2,3803,3671,8.0,...,13,0,5,2,1.100000,158.458333,256.833333,1.791667,0.458333,181.095238
4,4681,17,16,3,12,38,6,2987,2865,9.0,...,11,0,5,1,1.176471,124.458333,195.041667,2.625000,0.541667,175.705882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609405,7382,25,24,9,19,42,4,4728,4592,10.0,...,15,0,6,10,1.360000,157.600000,246.066667,3.866667,0.500000,189.120000
609406,6372,22,22,4,15,45,6,4311,4259,20.0,...,15,0,6,1,1.130435,143.700000,212.400000,2.800000,0.500000,195.954545
609407,8078,29,23,11,18,67,3,5266,4313,10.0,...,15,0,6,10,1.666667,175.533333,269.266667,4.266667,0.500000,181.586207
609408,9031,32,19,2,18,25,3,5531,3720,34.0,...,15,0,6,3,1.700000,184.366667,301.033333,2.100000,0.500000,172.843750


### Standardize with StandardScaler()

In [17]:
df_standard = df_clean[['stats_score', 'stats_kills', 'stats_deaths', 'stats_assists',
       'stats_headshots', 'stats_bodyshots', 'stats_legshots',
       'stats_damage_dealt', 'stats_damage_received', 'ability_casts_grenade',
       'ability_casts_ability1', 'ability_casts_ability2',
       'ability_casts_ultimate', 'tier_id', 'account_level', 'economy_spent_overall',
       'economy_spent_average', 'economy_loadout_value_overall',
       'economy_loadout_value_average', 'total_plants', 'total_defuses',
       'rounds_won', 'rounds_lost', 'total_rounds','KDA Ratio', 'ADR', 'ACS',
       'Overall Ability Cast', 'Win Ratio', 'damage_per_kills']]

df_cats = df_clean[['queue_id_numeric','map_name_numeric', 'agent_name_numeric']]

In [18]:
df_cats.shape

(382940, 3)

In [19]:
numeric_cols = df_standard.select_dtypes(include=['number'])
scaler = StandardScaler()

df_scaled = pd.DataFrame(scaler.fit_transform(numeric_cols), columns=numeric_cols.columns)

In [20]:
df_all = pd.merge(df_scaled, df_cats, left_index=True, right_index=True)
df_all

,stats_score,stats_kills,stats_deaths,stats_assists,stats_headshots,stats_bodyshots,stats_legshots,stats_damage_dealt,stats_damage_received,ability_casts_grenade,...,total_rounds,KDA Ratio,ADR,ACS,Overall Ability Cast,Win Ratio,damage_per_kills,queue_id_numeric,map_name_numeric,agent_name_numeric
0,-0.373784,-0.418093,0.856048,0.503302,-0.081188,-0.578460,-0.901913,-0.344157,1.354924,0.243616,...,0.793190,-0.551241,-0.719950,-0.731170,0.688671,-0.215257,0.112991,0,5,1
1,0.828314,0.752924,0.856048,0.800652,1.192798,-0.516386,0.068347,0.802564,0.643662,0.042060,...,0.793190,0.127553,0.442102,0.471817,-1.252211,0.274613,-0.176264,0,5,4
2,-0.067874,0.083771,1.104480,-0.091399,-0.991177,0.787160,2.655705,-0.221738,0.682217,1.553728,...,0.793190,-0.551241,-0.595894,-0.425034,1.076847,-0.215257,-0.757141,0,5,7
3,1.052361,0.920212,0.856048,-1.280799,0.646804,-0.454312,-0.255073,0.816910,0.960074,-0.461829,...,0.793190,-0.381543,0.456640,0.696029,-0.669947,-0.215257,-0.422546,0,5,2
4,0.139554,0.251059,0.110753,-0.686099,-0.081188,0.414718,1.038606,0.036489,-0.111472,-0.361052,...,0.793190,-0.251773,-0.334214,-0.217452,0.300494,0.274613,-0.582370,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382912,-0.319619,-0.752670,1.352911,-0.091399,-1.173175,0.228498,-0.578493,-0.256169,0.527999,-0.562607,...,0.238629,-1.014056,-0.394568,-0.456616,-0.744935,-0.504725,1.434838,1,4,10
382913,0.634427,0.251059,-0.137679,0.205952,-0.263186,3.828768,2.332285,0.266981,0.024133,0.142838,...,0.238629,0.191190,0.183771,0.584931,0.472527,0.564082,-0.161960,1,4,19
382914,-0.060488,0.083771,0.110753,-0.091399,-0.081188,-0.019797,-0.255073,0.144562,0.637015,-0.058718,...,0.238629,-0.151950,0.048437,-0.173718,-0.056804,0.564082,-0.047263,1,4,6
382915,-0.657536,-0.585381,-0.634542,0.800652,-0.263186,-1.137122,-0.901913,-0.529699,-0.552853,-0.764163,...,0.238629,0.176039,-0.696953,-0.825525,-0.374403,0.564082,0.125728,1,4,23


In [23]:
df_all.to_csv(OUTPUT_DIR + 'df_all.csv',index=False)